In [22]:
from src.experiment_based_function import SEEDS
import pandas as pd
%matplotlib inline

In [23]:
import os
import glob


In [24]:
eval_or_grid='eval'

In [25]:
def get_exp_result(eval_or_grid='eval'):
    res = []    
    for fn in glob.glob('experiment_1001/exp10-34class/*/*/*/%s_res.csv' % eval_or_grid):
        _, lvl, i_thres, seed, _ = fn.split(os.path.sep)
        seed = seed.replace('seed_','')
        df = pd.read_csv(fn, index_col=0)
        df['lvl'] = lvl
        df['seed'] = seed
        df['i_thres']=i_thres
        res.append(df)
    return pd.concat(res)


In [26]:
eval_res = get_exp_result('eval')

In [27]:
col_mapping = {' ': ' ',
 'RoadNet': 'RoadNet+Spatial',
 'Segment_NO_TOTAL_~2014+Spatial': 'Social type+Spatial', 
 'Segment_TOTAL_~2014+Spatial': 'Social total+Spatial',
 'RoadNet+Segment_NO_TOTAL_~2014+Spatial': 'RoadNet+Social type+Spatial', 
 'RoadNet+Segment_TOTAL_~2014+Spatial': 'RoadNet+Social total+Spatial'}


In [28]:
eval_res.ftr_combo_name = eval_res.ftr_combo_name.apply(lambda x: col_mapping[x.split('-')[0]] if x.split('-')[0] in col_mapping else x.split('-')[0])

In [29]:
eval_res.columns, eval_res.shape

(Index(['#ftr_all', '#ftr_keep', '#test_sample', '#train_sample',
        'feature_selection', 'ftr_combo_name', 'model_name', 'test_acc',
        'test_f1_macro', 'test_f1_micro', 'test_f1_weighted', 'test_mse',
        'test_n_classes', 'test_precision_macro', 'test_precision_micro',
        'test_precision_weighted', 'test_recall_macro', 'test_recall_micro',
        'test_recall_weighted', 'train_acc', 'train_f1_macro', 'train_f1_micro',
        'train_f1_weighted', 'train_mse', 'train_n_classes',
        'train_precision_macro', 'train_precision_micro',
        'train_precision_weighted', 'train_recall_macro', 'train_recall_micro',
        'train_recall_weighted', 'upsample', 'y_dist', 'y_dist_up', 'lvl',
        'seed', 'i_thres'],
       dtype='object'), (1440, 37))

In [30]:
print('train', eval_res.train_n_classes.value_counts().to_dict())
print('test', eval_res.test_n_classes.value_counts().to_dict())


print('train', 'test')
print(eval_res.apply(lambda x: (x.train_n_classes, x.test_n_classes), axis=1).value_counts())

train {4: 745, 3: 695}
test {4: 745, 3: 695}
train test
(4, 4)    745
(3, 3)    695
dtype: int64


In [31]:
f1_choices = ['test_f1_weighted', 'test_f1_macro', 'test_f1_micro']

In [32]:
from IPython.display import display

In [33]:
def get_paper_table(means, f1_type, key_cols, i):

    means_for_pivot = means.reset_index()
    means_for_pivot['key'] = means_for_pivot.apply(lambda x: tuple([x[col] for col in key_cols]), axis=1)
    table = means_for_pivot.reset_index().pivot(index='key', columns='ftr_combo_name', values=f1_type)
    table.index = pd.MultiIndex.from_tuples(table.index, names=key_cols)    
    table.columns.name = f1_type
    table = table[['RoadNet+Social type+Spatial']]
    table.index.name=None
    table.columns = [i]
    table.columns.name= f1_type
    return table

In [34]:
keys = ['model_name', 'i_thres', 'lvl', 'ftr_combo_name', 'feature_selection', 'upsample']
means = eval_res.groupby(keys).mean()[f1_choices+['#ftr_keep', '#ftr_all']]


In [14]:
i = '0.00'
lvl = '3lvl'
slice_i = means.iloc[(means.index.get_level_values(1) == i) & (means.index.get_level_values(2)==lvl)]
print(i, lvl, slice_i.test_f1_macro.max(), slice_i.test_f1_micro.max())
macro = get_paper_table(slice_i, f1_choices[1], ['model_name', 'feature_selection', 'upsample'], i)
micro = get_paper_table(slice_i, f1_choices[2], ['model_name', 'feature_selection', 'upsample'],i )

0.00 3lvl 0.632801404292 0.884269662921


In [38]:
f1_type = f1_choices[1]
lvl = '4lvl'
print(lvl, f1_type)
f1 = pd.DataFrame()
for i in [0, 0.1, 0.3, 0.5, 0.7, 0.8]:
    i = '%0.2f' % i
    slice_i = means.iloc[(means.index.get_level_values(1) == i) & (means.index.get_level_values(2)==lvl)]
    df = get_paper_table(slice_i, f1_type, ['model_name', 'feature_selection', 'upsample'], i)
    if f1.empty:
        f1 = df
    else:
        f1 = f1.join(df)

f1.fillna('')

4lvl test_f1_macro


test_f1_macro                              0.00      0.10      0.30      0.50  \
model_name feature_selection upsample                                           
BAGcls     None              None      0.354479  0.354244  0.353412  0.365158   
                             regular   0.478499                                 
                             svm       0.490870  0.479824  0.503318  0.453512   
           mrmr              regular   0.491639                                 
                             svm       0.422649                                 
           rfecv_linsvc      None      0.359398                                 
                             regular   0.469955                                 
                             svm       0.434624                                 
GDBcls     None              None      0.408893  0.443709  0.418327  0.415716   
                             regular   0.504843  0.490673  0.467571  0.483767   
                             svm       0.436686                                 
           mrmr              regular   0.484473                                 
                             svm       0.415022                                 
           rfecv_linsvc      None      0.395383                                 
                             regular   0.486292                                 
                             svm       0.430564                                 
RFcls      None              None      0.333275  0.361044  0.377734  0.364576   
                             regular   0.491658                                 
                             svm       0.432194                                 
           mrmr              regular   0.494311                                 
                             svm       0.438432                                 
           rfecv_linsvc      None      0.373081                                 
                             regular   0.473471                                 
                             svm       0.420229                                 
SVM        None              None      0.408251  0.475226  0.468307  0.483114   
                             regular   0.510019                                 
                             svm       0.493808                                 
           mrmr              regular   0.508956                                 
                             svm       0.493778                                 
           rfecv_linsvc      None      0.456432                                 
                             regular   0.498689                                 
                             svm       0.473143                                 
XGBcls     None              None      0.415348  0.423533  0.398981  0.396175   
                             regular   0.457232                                 
                             svm       0.430297                                 
           mrmr              regular   0.437724                                 
                             svm       0.440979                                 
           rfecv_linsvc      None      0.424212                                 
                             regular   0.448542                                 
                             svm       0.481870                                 

test_f1_macro                              0.70      0.80  
model_name feature_selection upsample                      
BAGcls     None              None      0.397384   0.35594  
                             regular                       
                             svm       0.467096  0.481972  
           mrmr              regular                       
                             svm                           
           rfecv_linsvc      None                          
                             regular                       
                             svm                         

In [45]:
i = '0.00'
lvl = '4lvl'
slice_i = means.iloc[(means.index.get_level_values(1) == i) & (means.index.get_level_values(2)==lvl)]
print(i, lvl, slice_i.test_f1_macro.max(), slice_i.test_f1_micro.max())
macro = get_paper_table(slice_i, f1_choices[1], ['model_name', 'feature_selection', 'upsample'])
micro = get_paper_table(slice_i, f1_choices[2], ['model_name', 'feature_selection', 'upsample'])
macro.join(micro)

0.00 4lvl 0.510018810126 0.695505617978


RoadNet+Social type+Spatial            test_f1_macro  test_f1_micro
model_name feature_selection upsample                              
BAGcls     None              None           0.354479       0.666292
                             regular        0.478499       0.683146
                             svm            0.490870       0.695506
           mrmr              regular        0.491639       0.692135
                             svm            0.422649       0.678652
           rfecv_linsvc      None           0.359398       0.655056
                             regular        0.469955       0.675406
                             svm            0.434624       0.671910
RFcls      None              None           0.333275       0.660674
                             regular        0.491658       0.689888
                             svm            0.432194       0.673034
           mrmr              regular        0.494311       0.683146
                             svm            0.438432       0.674157
           rfecv_linsvc      None           0.373081       0.661798
                             regular        0.473471       0.677903
                             svm            0.420229       0.674157
SVM        None              None           0.408251       0.661798
                             regular        0.510019       0.676404
                             svm            0.493808       0.669663
           mrmr              regular        0.508956       0.669663
                             svm            0.493778       0.662921
           rfecv_linsvc      None           0.456432       0.635955
                             regular        0.498689       0.644195
                             svm            0.473143       0.647191
XGBcls     None              None           0.415348       0.678652
                             regular        0.457232       0.682022
                             svm            0.430297       0.682022
           mrmr              regular        0.437724       0.671910
                             svm            0.440979       0.664045
           rfecv_linsvc      None           0.424212       0.653933
                             regular        0.448542       0.680400
                             svm            0.481870       0.685393